# Data Cleaning

## Imports

In [10]:
import numpy as np
import pandas as pd 
from numpy import NaN
import re
from datetime import datetime
from dateutil import parser
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

invalid = 'INVALID'
client_excel = r"../DATA-Excel/gis clients list 18-Jan-2022.csv"

- Clients dataframe

In [11]:
columns = ['CLNT_CODE', 'CLIENT_NAME', 'CLNT_ID_REG_NO', 'CLNT_DOB', 'CLNT_PIN', 'CLNT_TYPE', 'CLNT_EMAIL_ADDRS', 'CLNT_TEL', 'CLNT_TEL2', 'CLNT_SMS_TEL', 'CLNT_TITLE', 'CLNT_GENDER']
clientdf = pd.read_csv(client_excel, encoding='unicode_escape', low_memory=False, usecols=columns)
df_bak = pd.read_csv(client_excel, encoding='unicode_escape', low_memory=False, usecols=columns)

In [12]:
clientdf.columns

Index(['CLIENT_NAME', 'CLNT_CODE', 'CLNT_ID_REG_NO', 'CLNT_DOB', 'CLNT_PIN',
       'CLNT_EMAIL_ADDRS', 'CLNT_TEL', 'CLNT_TEL2', 'CLNT_TYPE', 'CLNT_TITLE',
       'CLNT_SMS_TEL', 'CLNT_GENDER'],
      dtype='object')

### Sample dataframe with edge cases

## ID registration no.

In [14]:
# null entries
null_dobs = clientdf['CLNT_ID_REG_NO'].isnull().sum().sum()
print('CLNT_ID_REG_NO Null Entries', '{:,}'.format(null_dobs))
# not null entries
dobs = clientdf['CLNT_ID_REG_NO'].dropna().count()
print('CLNT_ID_REG_NO Non null etries', '{:,}'.format(dobs))
# all clients
print('All clients','{:,}'.format(clientdf['CLNT_CODE'].count()))

print(dobs + null_dobs == clientdf['CLNT_CODE'].count(), ',(null values + non-null values) == count of all clients')


CLNT_ID_REG_NO Null Entries 75,431
CLNT_ID_REG_NO Non null etries 43,356
All clients 118,787
True ,(null values + non-null values) == count of all clients


In [15]:
# [x if y in ['I', 'S', 'F'] else 'OK' for x, y in zip(clientdf.CLNT_ID_REG_NO, clientdf.CLNT_TYPE)]

In [16]:
def id_reg_validator(df):
    
    '''Accepts a list or series of id as a 
    ids = clientdf['CLNT_ID_REG_NO']'''

    copydf = df.copy(deep=True)
    
    ids = [x if y in ['I', 'S', 'F'] else 'OK' for x, y in zip(copydf.CLNT_ID_REG_NO, copydf.CLNT_TYPE)]
    # Fill null values with INVALID
    # ids.fillna(invalid, inplace=True)
    try:
        for j in range(len(ids)):
            #convert id to str if it's not already
            if type(ids[j]) != str:
                ids[j] = str(ids[j]).strip()
            #remove trailing spaces    
            else:
                ids[j] =  ids[j].strip().replace(' ', '')
            # check the length of the string is either 7 or 8 (6??)
            if ids[j] == 'OK':
                pass #continue
            #check that id only contains digits
            elif re.match(r"^[0-9]+$", ids[j]) == None:
                # if re.match(r"^[A]{1}[K]{1}[0-9]+$", ids[j]) == None:
                # check if ID is passport number which is alphanumeric
                if re.match(r"^[A-Z]+[0-9]+$", ids[j]) == None:
                    ids[j] = invalid
                else:
                    if len(ids[j]) < 9:
                        ids[j] == invalid
            else:
                if len(ids[j]) < 7 or len(ids[j]) > 8:
                    ids[j] = invalid
                else:
                    #check that ID is not a guess of consecutive numbers
                    digits_in_id = []
                    for i in range(len(ids[j])):
                        digits_in_id.append(int(ids[j][i]))
                    guesses = [digits_in_id] #two digit guesses , digits_in_id[1: -1: 2] three guesses id_numbers[2: -1: 3]
                    for numbers in guesses:
                        steps = len(numbers) - 1
                        if sum(np.diff(numbers) == 1) == steps or sum(np.diff(numbers) == 0) == steps:# reason for sorting the numbers? a more strict check
                            ids[j] = invalid
                        else:
                            pass
    except:
        pass

    df['ID_STATUS'] = [x if x == invalid else 'VALID' for x in ids]
  

## Email

In [17]:
# null entries
null_dobs = clientdf['CLNT_EMAIL_ADDRS'].isnull().sum().sum()
print('CLNT_EMAIL_ADDRS Null Entries', '{:,}'.format(null_dobs))
# not null entries
dobs = clientdf['CLNT_EMAIL_ADDRS'].dropna().count()
print('CLNT_EMAIL_ADDRS Non null etries', '{:,}'.format(dobs))
# all clients
print('All clients','{:,}'.format(clientdf['CLNT_CODE'].count()))

print(dobs + null_dobs == clientdf['CLNT_CODE'].count(), ',(null values + non-null values) == count of all clients')

CLNT_EMAIL_ADDRS Null Entries 77,923
CLNT_EMAIL_ADDRS Non null etries 40,864
All clients 118,787
True ,(null values + non-null values) == count of all clients


In [18]:
def email_tokenize(email):
    addresses = email.split('/')
    return addresses


In [19]:
# Email validation
# re.match(pattern, string, flags=0)
# username@domain.top-level-domain

def email_validator(df):
    emails = df['CLNT_EMAIL_ADDRS'].copy(deep=True)
    emails.fillna(invalid, inplace=True)
    try:
        for a in range(len(emails)):
            #type checking and conversion
            if type(emails[a]) != str:
                emails[a] = invalid
            if ' ' in emails[a]:
                emails[a] = emails[a].strip().replace(' ', '')
            # check if email is blank
            elif len(emails[a].strip().replace(' ', '')) == 0:
                emails[a] = invalid
            # check for two emails and save one
            elif emails[a].find('/') != -1:
                addresses = email_tokenize(emails[a])
                emails[a] = addresses[0]
                df['CLNT_EMAIL_ADDRS'][a] = addresses[0]
            # check validity of email format
            elif re.match(r"(^[a-zA-Z0-9.+_&-]+@[a-zA-Z0-9-.]+\.[a-zA-Z0-9-.]+$)", emails[a]) == None:
                try:
                    if re.match(r"(^[a-zA-Z0-9.+_&-]+@[a-zA-Z0-9-.]+\.[a-zA-Z0-9-.]+$)", emails[a].strip().replace(' ', '')) == None:
                        emails[a] = invalid
                except:
                    pass
            # if email is invalid because of spaces, remove space
            else:
                # if ' ' in emails[a]:
                #     emails[a] = emails[a].strip().replace(' ', '')
                continue
    except:
        pass
    df['EMAIL_STATUS'] = [x if x == invalid else 'VALID' for x in emails]
        




## Phone numbers

- Telephone 1

In [20]:
# null entries
null_dobs = clientdf['CLNT_TEL'].isnull().sum().sum()
print('CLNT_TEL Null Entries', '{:,}'.format(null_dobs))
# not null entries
dobs = clientdf['CLNT_TEL'].dropna().count()
print('CLNT_TEL Non null etries', '{:,}'.format(dobs))
# all clients
print('All clients','{:,}'.format(clientdf['CLNT_CODE'].count()))

print(dobs + null_dobs == clientdf['CLNT_CODE'].count(), ',(null values + non-null values) == count of all clients')

CLNT_TEL Null Entries 83,373
CLNT_TEL Non null etries 35,414
All clients 118,787
True ,(null values + non-null values) == count of all clients


- SMS Telephone 

In [21]:
def wrapper1(phone_numbers_list):
    phone_numbers_list.fillna(invalid, inplace=True)
    phone_len = [9, 10, 12, 13, 15, 16]
    try:
        for i in range(len(phone_numbers_list)):
            if type(phone_numbers_list[i]) != str:
                phone_numbers_list[i] = str(phone_numbers_list[i])

            if len(phone_numbers_list[i].replace(' ', '')) not in phone_len:
                phone_numbers_list[i] = invalid
            elif re.match(r"(^[+]*[(]{0,1}[0-9]{1,4}[)]{0,1}[\s]{0,1}[-\s\./0-9]*$)", phone_numbers_list[i]) == None:
                phone_numbers_list[i] = invalid
            elif phone_numbers_list[i][0] == '+' and len(phone_numbers_list[i].replace(' ', '')) not in [13, 15, 16]:
                phone_numbers_list[i] = invalid
            else:
                for j in ['(', '+', ' ', ')']:
                    if j in phone_numbers_list[i]:
                        phone_numbers_list[i] = phone_numbers_list[i].replace(j, '')

                    
    except:
        pass
    return [x if x == invalid else 'VALID' for x in phone_numbers_list]

def phone_num_validator(df):
    phone_numbers_list = df['CLNT_TEL'].copy(deep=True)
    sms_list = df['CLNT_SMS_TEL'].copy(deep=True)

    df['TEL_STATUS'] = wrapper1(phone_numbers_list)
    df['SMS_STATUS'] = wrapper1(sms_list)
    
        
        


In [22]:
def populate_tel(df):
    tel_status = df.CLNT_TEL
    sms_status = df.CLNT_SMS_TEL
    for i in range(len(tel_status)):
        try:
            if tel_status[i] == None and sms_status[i] == None:
                tel_status = sms_status = invalid
            elif tel_status[i] and sms_status[i] == None:
                sms_status[i] = tel_status[i]
            elif tel_status[i] == None and sms_status[i]:
                tel_status[i] = sms_status[i]
            else:
                pass

        except:
            raise




## Date of birth

In [23]:
# null entries
null_dobs = clientdf['CLNT_DOB'].isnull().sum().sum()
print('CLNT_DOB Null Entries', '{:,}'.format(null_dobs))
# not null entries
dobs = clientdf['CLNT_DOB'].dropna().count()
print('CLNT_DOB Non null etries', '{:,}'.format(dobs))
# all clients
print('All clients','{:,}'.format(clientdf['CLNT_CODE'].count()))

print(dobs + null_dobs == clientdf['CLNT_CODE'].count(), ',(null values + non-null values) == count of all clients')

CLNT_DOB Null Entries 111,410
CLNT_DOB Non null etries 7,377
All clients 118,787
True ,(null values + non-null values) == count of all clients


## Age

In [24]:
# epoch = 1970 
# import time
# time.gmtime(0)
# https://docs.python.org/3/library/time.html#:~:text=Function%20strptime()%20can%20parse%202%2Ddigit%20years%20when%20given%20%25y%20format%20code.%20When%202%2Ddigit%20years%20are%20parsed%2C%20they%20are%20converted%20according%20to%20the%20POSIX%20and%20ISO%20C%20standards%3A%20values%2069%E2%80%9399%20are%20mapped%20to%201969%E2%80%931999%2C%20and%20values%200%E2%80%9368%20are%20mapped%20to%202000%E2%80%932068.


In [25]:
def age(dob):
    # print(type(dob),'dob = ', dob)
    age = datetime.today().year - datetime.strptime(datetime.strftime(dob, '%d-%b-%Y'), '%d-%b-%Y').year
    return age

In [26]:
x=datetime.strptime('4/28/1949', '%m/%d/%Y')
datetime.today().year - datetime.strptime(datetime.strftime(x, '%d-%b-%Y'), '%d-%b-%Y').year

73

In [27]:
datetime.strptime(datetime.strftime(x, '%d-%b-%Y'), '%d-%b-%Y').year

1949

In [28]:
import time
time.gmtime(0)

time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=3, tm_yday=1, tm_isdst=0)

In [29]:
# check for different formats used in DOB column. set() returning one value means that one format is used.
# dob_formats = ['%m/%d/%Y like so 01/21/1987' if re.match(r'^(1[0-2]|0?[1-9])/(3[01]|[12][0-9]|0?[1-9])/(?:[0-9]{2})?[0-9]{2}$', i) else i for i in clientdf['CLNT_DOB'].dropna()]
# print('Formats: {}\n {}'.format(len(set(dob_formats)), [i for i in set(dob_formats)] ))



def dob_validator(df):
    '''Takes a list or series of dates and flags as invalid wrong dates.'''

    types = df['CLNT_TYPE'].copy(deep=True)
    dates = df['CLNT_DOB'].copy(deep=True)
    dates = pd.Series(['VALID' if y in ['C', 'P', 'G', 'M'] else x for x,y in zip(dates, types)])

    dates.fillna(invalid, inplace=True)


    if type(dates) == str:
        dates = list(dates)
    try:
        for i in range(len(dates)):
            if type(dates[i]) != str:
                dates[i] = str(dates[i])
            if dates[i] == 'VALID' or dates[i] == 'INVALID':
                pass

            else:
                try:
                    datetime.strptime(dates[i], '%m/%d/%Y')
                except (Exception): # ...does not match format '%m/%d/%Y'   
                        try:
                            dob_date = str(parser.parse(dates[i]))
                            # validate it's a date before assigning
                            dates[i] = datetime.strftime(datetime.strptime(dob_date, '%Y-%m-%d %H:%M:%S'), '%m/%d/%Y')
                        except (Exception):
                                dates[i] = invalid
                else:
                    if re.match(r'^(1[0-2]|0?[1-9])/(3[01]|[12][0-9]|0?[1-9])/(?:[0-9]{2})?[0-9]{2}$', dates[i]) == None:
                        dates[i] == invalid
    except:
        pass
    df['DOB_STATUS'] = [x if x == invalid else 'VALID' for x in dates]
    df['AGE'] = [age(datetime.strptime(dob, '%m/%d/%Y')) if (dob != invalid and dob != 'VALID') else 0 for dob in dates]

    # df['DOB_STATUS'] = [x if (x > 18 or x < 100) else invalid for x in df['AGE']]
    # df['AGE_STATUS'] = [x if (x > 18 or x < 100) else invalid for x in df['AGE']]

    status = []
    for i in range(len(df['AGE'])):
        if df['AGE'][i] == 0 and types[i] not in ['C', 'P', 'G', 'M']:
            status.append(invalid)
        elif (df['AGE'][i] < 18 or df['AGE'][i] > 100) and types[i] not in ['C', 'P', 'G', 'M']:
            status.append(invalid)
        else:
            status.append('VALID')
    df['AGE_STATUS'] = status

## SMS telephone

## Client PIN

In [30]:
#check that pin is not a guess of consecutive numbers or same number
def consecutive_or_dup_pin_numbers(pin):
    if type(pin) != str:
        pin = str(pin)
    # print('pin = ', pin)
    num = pin[1:-1]
    digits_in_num = []
    for i in range(len(num)):
        digits_in_num.append(int(num[i]))
    # print('digits_in_num = ',digits_in_num)
    guesses = [digits_in_num] #two digit guesses , digits_in_id[1: -1: 2] three guesses id_numbers[2: -1: 3]
    # print('guesses', guesses)
    for numbers in guesses:
        # print('numbers = ', numbers)
        steps = len(numbers) - 1
        if sum(np.diff(numbers) == 1) == steps or sum(np.diff(numbers) == 0) == steps:# A222222222K and A123456789K fail
            return True
        elif len(set(numbers)) in [1, 2]:# A0000000001K fails
            return True
        else:
            return False

In [31]:
def kra_pin_validator(clientdf):
    
    pins = clientdf['CLNT_PIN'].copy(deep=True)
    types = clientdf['CLNT_TYPE'].copy(deep=True)
    
    pins.fillna(invalid, inplace=True)
    # types.fillna(invalid, inplace=True)

    try:
        for i in range(len(pins)):
            if type(pins[i]) != str:
                pins[i] = str(pins[i])

            if len(pins[i]) != 11:
                pins[i] = invalid
            # if type(types[i]) != str:
            #         types[i] = str(types[i])

            # if types[i] not in ['I', 'F','P', 'S', 'M', 'C', 'G']:
            #     types[i] = invalid
            
            # PIN is alphanumeric
            if re.match(r"^[AP]{1}[0-9]+[A-Z]{1}$", pins[i].replace(' ', '')) == None:
                if re.match(r"^[apAP]{1}[0-9]+[A-Za-z]{1}$", pins[i].replace(' ', '')) == None:
                    pins[i] = invalid
                else: #convert to upper valid pins with lower case chars
                    pins[i] = pins[i].upper()
                    clientdf['CLNT_PIN'][i] = pins[i].upper()
                    if consecutive_or_dup_pin_numbers(pins[i]) == True:
                        pins[i] = invalid
                    else:
                        continue
            else: #check client type corresponds to first letter of kra pin
                chars = {'A': ['I', 'F', 'S', 'M'], 'P': ['C', 'G', 'M', 'P']}
                if consecutive_or_dup_pin_numbers(pins[i]) == True:
                    pins[i] = invalid
                elif types[i] not in chars[pins[i][0]]:
                    pins[i] = invalid

                else:
                    pass

    except:
        pass
    clientdf['PIN_STATUS'] = [x if x == invalid else 'VALID' for x in pins]
    clientdf['TYPE_STATUS'] = [x if x == invalid else 'VALID' for x in types]

- bypass

In [32]:
# [(pin, consecutive_or_dup_pin_numbers(pin)) for pin in bypass]

### Client Type

In [33]:
# null entries
null_dobs = clientdf['CLNT_TYPE'].isnull().sum().sum()
print('CLNT_DOB Null Entries', '{:,}'.format(null_dobs))
# not null entries
dobs = clientdf['CLNT_TYPE'].dropna().count()
print('CLNT_DOB Non null etries', '{:,}'.format(dobs))
# all clients
print('All clients','{:,}'.format(clientdf['CLNT_CODE'].count()))

print(dobs + null_dobs == clientdf['CLNT_CODE'].count(), ',(null values + non-null values) == count of all clients')

CLNT_DOB Null Entries 1,605
CLNT_DOB Non null etries 117,182
All clients 118,787
True ,(null values + non-null values) == count of all clients


## Gender

In [34]:

display(len(set(clientdf['CLNT_TITLE'])))
x = list(set(clientdf['CLNT_TITLE']))
x.remove(NaN)
x

41

['REV.',
 'LADY',
 'MASTER',
 'Miss',
 'ALHAJI',
 'PASTOR',
 'SIR',
 'DOCTOR',
 'Mr',
 'MRS.',
 'MS',
 'SURV.',
 'Mrs',
 'HON.',
 'BISHOP',
 'BARRISTER',
 'MS.',
 'MISS.',
 'CAPTAIN',
 'PROF.',
 'ENGR.',
 'MRS',
 'MISS',
 'Mr.',
 'REV. (MRS)',
 'MR & MRS',
 'AMBASADOR',
 'DR.',
 'ALHAJA',
 'MR.',
 'ENGINEER',
 'Dr',
 'Prof',
 'Ms',
 'REV. FR',
 'PROFESSOR',
 'HON. JUSTICE',
 'EST. OF LATE',
 'CAPT.',
 'ARCHITECT']

In [35]:
clientdf.columns

Index(['CLIENT_NAME', 'CLNT_CODE', 'CLNT_ID_REG_NO', 'CLNT_DOB', 'CLNT_PIN',
       'CLNT_EMAIL_ADDRS', 'CLNT_TEL', 'CLNT_TEL2', 'CLNT_TYPE', 'CLNT_TITLE',
       'CLNT_SMS_TEL', 'CLNT_GENDER'],
      dtype='object')

In [36]:
from numpy import NaN

def gender_parser(df):
    titles_in_data = list(set(df['CLNT_TITLE']))
    titles_in_data.remove(NaN)
    
    female =  ['LADY','Miss','MRS.','MS','Mrs','MS.','MISS.', 'MRS','REV. (MRS)','Ms','MISS']
    male = ['MASTER','SIR','Mr','Mr.','MR.','REV. FR']
    unisex = ['REV.', 'ALHAJI', 'PASTOR','DOCTOR','SURV.', 'HON.','BISHOP','BARRISTER', 'CAPTAIN','PROF.','ENGR.',
                'MR & MRS','AMBASADOR','DR.','ALHAJA', 'ENGINEER','Dr','Prof', 'PROFESSOR','HON. JUSTICE','EST. OF LATE','CAPT.','ARCHITECT']

    types = df['CLNT_TYPE'].copy(deep=True)
    titles = df['CLNT_TITLE'].copy(deep=True)
    gender = df['CLNT_GENDER'].copy(deep=True)

    titles = pd.Series(['VALID' if y in ['C', 'P', 'G', 'M'] else x for x, y in zip(titles, types)])
    gender = pd.Series(['VALID' if y in ['F', 'M'] else x for x, y in zip(titles, gender)])
    
    titles = pd.Series(['VALID' if (y == 'VALID' or x == 'VALID') else x for x, y in zip(titles, gender)])

    titles.fillna(invalid, inplace=True)
    
    for i in range(len(titles)):
        if titles[i] != str:
            titles[i] = str(titles[i])
        if titles[i] == 'VALID' or titles[i] == 'INVALID':
            pass
        else:
            if titles[i] in unisex:
                titles[i] = 'UNISEX TITLE'
            elif titles[i] in male:
                titles[i] = 'MALE'
            elif titles[i] in female:
                titles[i] = 'FEMALE'
            else:
                titles[i] = invalid
                
    df['GENDER_STATUS'] = [invalid if x in [invalid, 'UNISEX TITLE'] else 'VALID' for x in titles]



    

# CLEANING

In [37]:
dob_validator(clientdf)
print('dob done')
phone_num_validator(clientdf)
print('telephones done')
id_reg_validator(clientdf )
print('IDs done')
email_validator(clientdf)
print('emails done')
kra_pin_validator(clientdf)
print('kra pins done')
gender_parser(clientdf)
print('gender done')

dob done
telephones done
IDs done
emails done
kra pins done
gender done


- Rearrange columns

In [38]:
clientdf = clientdf[['CLIENT_NAME', 'CLNT_CODE', 
                        'CLNT_ID_REG_NO', 'ID_STATUS', 
                        'CLNT_DOB', 'DOB_STATUS',
                        'AGE' ,'AGE_STATUS',
                        'CLNT_PIN','PIN_STATUS',
                        'CLNT_TYPE','TYPE_STATUS',
                        'CLNT_EMAIL_ADDRS', 'EMAIL_STATUS', 
                        'CLNT_TEL', 'TEL_STATUS', 
                        'CLNT_SMS_TEL','SMS_STATUS',
                        'CLNT_TITLE', 'CLNT_GENDER','GENDER_STATUS'
                    
                    ]]

- New CSV, xlsx

In [39]:
# clientdf.to_excel('Cleaned-client-excel-v1.xlsx', engine='openpyxl', index=False)
# clientdf.to_csv('Cleaned-client-data-v5.csv', index=False)


- xlsxwriter new workbook + Add Color to it

    runs in 1 min 30 sec

In [40]:
import xlsxwriter
wb = xlsxwriter.Workbook('Gis-Data-validity-TRIAL.xlsx')
worksheet = wb.add_worksheet()

cell_format = wb.add_format({'border': 1})
cell_format.set_bg_color('#F9E79F')
cell_format.set_border_color('#000000')

cell_format1 = wb.add_format({'border': 1})
cell_format1.set_bg_color('#F5CBA7')
cell_format1.set_border_color('#000000')

cell_format2 = wb.add_format({'border': 1})
cell_format2.set_bg_color('#FAD7A0')
cell_format2.set_border_color('#000000')

cell_format3 = wb.add_format({'border': 1})
cell_format3.set_bg_color('#76D7C4')
cell_format3.set_border_color('#000000')

cell_format4 = wb.add_format({'border': 1})
cell_format4.set_bg_color('#D6EAF8')
cell_format4.set_border_color('#000000')

cell_format5 = wb.add_format({'border': 1})
cell_format5.set_bg_color('#E8DAEF')
cell_format5.set_border_color('#000000')

cell_format6 = wb.add_format({'border': 1})
cell_format6.set_bg_color('#D5F5E3')
cell_format6.set_border_color('#000000')

cell_format7 = wb.add_format({'border': 1})
cell_format7.set_bg_color('#FEF9E7')
cell_format7.set_border_color('#000000')


cell_format8 = wb.add_format({'border': 1})
cell_format8.set_bg_color('#B9C8FD')
cell_format8.set_border_color('#000000')


for k in range(len(clientdf.columns)):
    for v in  range(len(clientdf['CLNT_ID_REG_NO'])):
        try:
            if clientdf.columns[k] in ['ID_STATUS']:
                worksheet.write(0, k, clientdf.columns[k], cell_format)
                worksheet.write(v + 1, k, clientdf[clientdf.columns[k]][v], cell_format)
            elif clientdf.columns[k] in ['DOB_STATUS']:
                worksheet.write(0, k, clientdf.columns[k], cell_format1)
                worksheet.write(v + 1, k, clientdf[clientdf.columns[k]][v], cell_format1)
            elif clientdf.columns[k] in ['PIN_STATUS']:
                worksheet.write(0, k, clientdf.columns[k], cell_format2)
                worksheet.write(v + 1, k, clientdf[clientdf.columns[k]][v], cell_format2)
            elif clientdf.columns[k] in ['TYPE_STATUS']:
                worksheet.write(0, k, clientdf.columns[k], cell_format4)
                worksheet.write(v + 1, k, clientdf[clientdf.columns[k]][v], cell_format4)
            elif clientdf.columns[k] in ['EMAIL_STATUS']:
                worksheet.write(0, k, clientdf.columns[k], cell_format5)
                worksheet.write(v + 1, k, clientdf[clientdf.columns[k]][v], cell_format5)
            elif clientdf.columns[k] in ['TEL_STATUS']:
                worksheet.write(0, k, clientdf.columns[k], cell_format6)
                worksheet.write(v + 1, k, clientdf[clientdf.columns[k]][v], cell_format6)
            elif clientdf.columns[k] in ['SMS_STATUS']:
                worksheet.write(0, k, clientdf.columns[k], cell_format7)
                worksheet.write(v + 1, k, clientdf[clientdf.columns[k]][v], cell_format7)
            elif clientdf.columns[k] in ['AGE_STATUS']:
                worksheet.write(0, k, clientdf.columns[k], cell_format6)
                worksheet.write(v + 1, k, clientdf[clientdf.columns[k]][v], cell_format6)
            elif clientdf.columns[k] in ['GENDER_STATUS']:
                worksheet.write(0, k, clientdf.columns[k], cell_format8)
                worksheet.write(v + 1, k, clientdf[clientdf.columns[k]][v], cell_format8)
            else:
                worksheet.write(0, k, clientdf.columns[k])
                worksheet.write(v + 1, k, clientdf[clientdf.columns[k]][v])
        except:
            pass

wb.close()